In [1]:
from sys import stdout
 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math

import shap
 
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import scale 

from keras.models import Sequential
from keras.layers import Dense

from progressbar import ProgressBar

In [2]:
#import training datadet
#import internal data
inter = pd.ExcelFile('training data_Github.xlsx')
inter = pd.read_excel(inter,'Combined')
#print(inter.info())
comb = inter.dropna(axis=0)
#print(comb.info())
comb = comb[comb["tR (min)"] != "?"]
#preprocess
for col in comb.columns[1:]:
    if (comb[col] == 0).all():
        comb = comb.drop([col], axis = 1)
comb = comb.reset_index(drop=True)
print(comb.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1790 entries, 0 to 1789
Columns: 145 entries, Project to fsulfone
dtypes: float64(101), int64(41), object(3)
memory usage: 2.0+ MB
None


In [3]:
#ANN portion


y = {'tR (min)': [round(x,4) for x in comb['tR (min)']]}
y = pd.DataFrame(comb, columns = ['tR (min)'])
X = comb.drop(['Project','Column','Compound','tR (min)'], axis=1)

### Sandardization of data ###
from sklearn.preprocessing import StandardScaler
PredictorScaler=StandardScaler()
TargetVarScaler=StandardScaler()

# Storing the fit object for later reference
PredictorScalerFit=PredictorScaler.fit(X)
TargetVarScalerFit=TargetVarScaler.fit(y)

# Generating the standardized values of X and y
X=PredictorScalerFit.transform(X)
y=TargetVarScalerFit.transform(y)

X = pd.DataFrame(X)
y = pd.DataFrame(y)
print(X.info())
print(y.info())

X_train = X
#Convert from dataframe to array
X_train = np.asarray(X_train).astype('float32')
y_train = np.asarray(y).astype('float32')


#ANN Regression
# create ANN model
model = Sequential()
# Defining the Input layer and FIRST hidden layer, both are same!
model.add(Dense(units=5, input_dim=141, kernel_initializer='normal', activation='relu'))
# Defining the Second layer of the model
# after the first layer we don't have to specify input_dim as keras configure it automatically
model.add(Dense(units=10, kernel_initializer='normal', activation='tanh'))
# Defining the Third layer of the model
model.add(Dense(units=10, kernel_initializer='normal', activation='tanh'))
# The output neuron is a single fully connected node 
# Since we will be predicting a single number
model.add(Dense(1, kernel_initializer='normal'))
# Compiling the model
model.compile(loss='mean_squared_error', optimizer='adam')    
# Fitting the ANN to the Training set
model.fit(X_train, y_train ,batch_size = 50, epochs = 50, verbose=0)

explainer = shap.KernelExplainer(model, X_train)
shap_values = explainer.shap_values(X_train)

import sys
size = sys.getsizeof(shap_values)
import csv
with open("shapley_ANN_re.csv", mode="w", newline="") as file:
    writer = csv.writer(file)
    
    writer.writerows(shap_values)



KeyError: "['Gnum'] not found in axis"